In [29]:
import pandas as pd
from nltk.metrics import edit_distance
import string
import numpy as np

In [7]:
df = pd.read_csv('tt_dataset.csv')
df

,text,phonemes,soundex
0,Peter Piper picked a peck of pickled peppers. ...,[P IY1 T ER0] [P AY1 P ER0] [P IH1 K T] [AH0] ...,"['P360', 'P160', 'P230', 'A000', 'P200', 'O100..."
1,I saw Susie sitting in a shoe shine shop. Wher...,[AY1] [S AO1] [S UW1 Z IY0] [S IH1 T IH0 NG] [...,"['I000', 'S000', 'S200', 'S352', 'I500', 'A000..."
2,How many boards Could the Mongols hoard If the...,[HH AW1] [M EH1 N IY0] [B AO1 R D Z] [K UH1 D]...,"['H000', 'M500', 'B632', 'C430', 'T000', 'M524..."
3,How can a clam cram in a clean cream can?,[HH AW1] [K AE1 N] [AH0] [K L AE1 M] [K R AE1 ...,"['H000', 'C500', 'A000', 'C450', 'C650', 'I500..."
4,Send toast to ten tense stout saints' ten tall...,[S EH1 N D] [T OW1 S T] [T UW1] [T EH1 N] [T E...,"['S530', 'T230', 'T000', 'T500', 'T520', 'S330..."
...,...,...,...
2671,Then step up mister and twist your tongue,"['DH', 'EH1', 'N', ' ', 'S', 'T', 'EH1', 'P', ...","['T500', 'S310', 'U100', 'M236', 'A530', 'T230..."
2672,Now Kissle will whistle at busty Miss. Russell...,"['N', 'AW1', ' ', 'K', 'IH1', 'S', 'AH0', 'L',...","['N000', 'K240', 'W400', 'W234', 'A300', 'B230..."
2673,"Purple paper people, purple paper people, purp...","['P', 'ER1', 'P', 'AH0', 'L', ' ', 'P', 'EY1',...","['P614', 'P160', 'P140', 'P614', 'P160', 'P140..."
2674,De doorgaans dappere Durgerdammer drukker Dirk...,"['D', 'IY1', ' ', 'D', 'AO1', 'R', 'G', 'AH0',...","['D000', 'D625', 'D160', 'D626', 'D626', 'D620..."


## implement metrics

In [15]:
### necessary functions

# remove all punctuation from a string
def remove_punctuation(input_string):
    punctuation_chars = string.punctuation # Create a string containing all punctuation marks
    no_punct = ''.join(char for char in input_string if char not in punctuation_chars) # Create new string without punctuation
    
    return no_punct



In [9]:
tongue_twister = df['text'][0]
print(tongue_twister)

Peter Piper picked a peck of pickled peppers. A peck of pickled peppers Peter Piper picked. If Peter Piper picked a peck of pickled peppers, Where's the peck of pickled peppers Peter Piper picked?


### for text

In [33]:
# calculates levenshtein dist between every two words in a string
# args: input string, boolean of allowing transpositions
def levenshtein_dist(tongue_twister, transp):
    words = remove_punctuation(tongue_twister).split(' ') # split tt into indiv words
    levenshtein_arr = np.zeros((len(words), len(words))) # initialise distance array
    for i in range(len(words)):
        for j in range(i+1, len(words)):
            dist = edit_distance(words[i], words[j], substitution_cost=1, transpositions=transp) # Transposition True because of difficukty to distinguish both words
            levenshtein_arr[i, j] = dist
            levenshtein_arr[j, i] = dist
            
    return levenshtein_arr



In [32]:
print(levenshtein_arr)

[[0. 2. 5. ... 0. 2. 5.]
 [2. 0. 4. ... 2. 0. 4.]
 [5. 4. 0. ... 5. 4. 0.]
 ...
 [0. 2. 5. ... 0. 2. 5.]
 [2. 0. 4. ... 2. 0. 4.]
 [5. 4. 0. ... 5. 4. 0.]]


### for phonemes

### for soundex